#  删除
Weaviate 允许通过 ID 或一组标准删除对象。

In [1]:
import json
import weaviate
from weaviate.auth import AuthApiKey

# 连接到本地部署的 Weaviate
client = weaviate.Client(
    url="http://127.0.0.1:8080",
    auth_client_secret=AuthApiKey("WVF5YThaHlkYwhGUSmCRgsX3tD5ngdN8pkih")
)

## 根据 ID 删除对象
要通过 id 删除，请指定集合名称和对象 id。

In [ ]:
## V3
uuid_to_delete = "..."  # replace with the id of the object you want to delete

client.data_object.delete(
    uuid=uuid_to_delete,
    class_name="EphemeralObject",  # Class of the object to be deleted
)

In [ ]:
## V4
uuid_to_delete = "..."  # replace with the id of the object you want to delete

collection = client.collections.get("EphemeralObject")
collection.data.delete_by_id(
    uuid_to_delete
)

## 删除多个对象
要删除符合一组条件的对象，请指定集合和where过滤器。

In [ ]:
## V3
client.batch.delete_objects(
    class_name="EphemeralObject",
    where={
        "path": ["name"],
        "operator": "Like",
        "valueText": "EphemeralObject*"
    },
)


In [ ]:
## V4
from weaviate.classes.query import Filter

collection = client.collections.get("EphemeralObject")
collection.data.delete_many(
    where=Filter.by_property("name").like("EphemeralObject*")
)

### 包含任何/包含全部
使用ContainsAny/ContainsAll过滤器根据一组标准删除对象。

In [ ]:
client.batch.delete_objects(
    class_name="EphemeralObject",
    where={
        "path": ["name"],
        "operator": "ContainsAny",
        "valueTextArray": ["asia", "europe"]  # Note the array syntax
    },
)

In [ ]:
from weaviate.classes.query import Filter

collection = client.collections.get("EphemeralObject")
collection.data.delete_many(
    where=Filter.by_property("name").contains_any(["europe", "asia"])
)

## 根据 id 删除多个对象
要根据 ID 值删除多个对象，请使用基于标准ContainsAny的过滤器（例如id） 

In [ ]:
### V3
client.batch.delete_objects(
    class_name="EphemeralObject",
    where={
        "path": ["id"],
        "operator": "ContainsAny",
        "valueTextArray": ["12c88739-7a4e-49fd-bf53-d6a829ba0261", "3022b8be-a6dd-4ef4-b213-821f65cee53b", "30de68c1-dd53-4bed-86ea-915f34faea63"]  # Note the array syntax
    },
)

In [ ]:
from weaviate.classes.query import Filter

collection = client.collections.get("EphemeralObject")

response = collection.query.fetch_objects(limit=3)  # Fetch 3 object IDs
ids = [o.uuid for o in response.objects]  # These can be lists of strings, or `UUID` objects

collection.data.delete_many(
    where=Filter.by_id().contains_any(ids)  # Delete the 3 objects
)

## 删除所有对象



###  可选参数
您可以使用它dryRun来检查有多少个对象将被删除，而无需实际执行删除操作。
设置output为'verbose'以查看每次删除的更多详细信息（ID 和删除状态）。

In [2]:
## V3
result = (
    client.batch.delete_objects(
        class_name="EphemeralObject",
        # Same `where` filter as in the GraphQL API
        where={
            "path": ["name"],
            "operator": "Like",
            "valueText": "EphemeralObject*"
        },
        dry_run=True,
        output="verbose"
    )
)

import json
print(json.dumps(result, indent=2))

UnexpectedStatusCodeException: Delete in batch! Unexpected status code: 422, with response body: {'error': [{'message': "validate: class: EphemeralObject doesn't exist"}]}.

In [ ]:
### V4
from weaviate.classes.query import Filter

collection = client.collections.get("EphemeralObject")
result = collection.data.delete_many(
    where=Filter.by_property("name").like("EphemeralObject*"),
    dry_run=True,
    verbose=True
)

print(result)